# Initialization

In [1]:
from patient import Patient
from openai import OpenAI
import pandas as pd

key = #api_key

In [2]:
df = pd.read_csv('RAG_dataset/new_42_dataset.csv')
df.head()

,output_text,input_text,sentence_1,sentence_2,sentence_3,sentence_4,sentence_5,sentence_6,sentence_7,sentence_8
0,acne,I've been having a really bad rash on my skin ...,I've been having a really bad rash on my skin ...,It's full of pus-filled pimples and blackheads.,My skin has also been scurring a lot.,no more information,NaN,NaN,NaN,
1,acne,I've just developed a severe rash on my skin. ...,I've just developed a severe rash on my skin.,It's clogged with pus-filled pimples and black...,My skin has also been quite sensitive.,no more information,NaN,NaN,NaN,
2,allergy,"My skin is itchy and red. Sometimes, it can pe...",My skin is itchy and red.,"Sometimes, it can peel.","My lips and cheeks swell, and it is really ann...",I sometimes get headaches and watery eyes beca...,no more information,NaN,NaN,
3,allergy,I have a sore throat and I am sneezing all the...,I have a sore throat and I am sneezing all the...,"Sometimes my face swells up, especially my lip...",I can't stop sneezing once I start.,no more information,NaN,NaN,NaN,
4,arthritis,I've been feeling really weak in my muscles an...,I've been feeling really weak in my muscles an...,My joints have been swelling up and it's hard ...,Walking has been really painful too.,no more information,NaN,NaN,NaN,


In [3]:
patient = Patient(key=key)

In [4]:
def run(i_min, doctor, full_histories, diagnoses_lists):
    # for i in range(i_min, min(i_min+5, len(df))):
    for i in range(i_min, 42):
        print(f'Patient {i+1}')

        n = 1
        full_history = []
        diagnoses_list = []
        threshold = 0.85
        if_continue = 1
        response = "Doctor: Hello! I'm your virtual doctor. Please describe your symptoms or concerns."

        symptoms = df.iloc[i]["input_text"]
        disease = df.iloc[i]["output_text"]

        print(f'Symptoms: {symptoms}')
        print(f'Disease: {disease}')
        print()

        while if_continue==1:
            if len(diagnoses_list) < 10:
                user_input = patient.reply(symptoms, disease, response)
            else:
                user_input = 'no more information'
                
            print(f'Patient: {user_input}\n')
            full_history.append(f"Patient: {user_input}")

            if_continue, diagnosis, response, full_history = doctor.ask_doctor(threshold, diagnoses_list, full_history)

            print(f'Doctor: {response}\n')
            diagnoses_list.append(diagnosis)
            n += 1

        full_histories.append(full_history)
        diagnoses_lists.append(diagnosis)
        print('\n------------------------------\n')
    
    return full_histories, diagnoses_lists
    

In [5]:
emb = pd.read_pickle('RAG_dataset/text_embeddings2.pkl')
df_embed = pd.read_csv('RAG_dataset/text_embeddings2.csv')

df_embed['embeddings'] = emb
df_embed.head()

,text,embedding,embeddings
0,"Abdominal Hernia: Visible bulge or swelling, d...","[-0.032867081463336945, 0.044648800045251846, ...","[-0.032867081463336945, 0.044648800045251846, ..."
1,"Abdominal Injury: Abdominal pain, tenderness, ...","[-0.02714613638818264, 0.06802134960889816, 0....","[-0.02714613638818264, 0.06802134960889816, 0...."
2,"Abscess: Swelling, pain, redness, fever, pus f...","[-0.006426552776247263, 0.021023059263825417, ...","[-0.006426552776247263, 0.021023059263825417, ..."
3,"Abscess of Nose: Pain, swelling, redness, disc...","[-0.018036143854260445, 0.03526167571544647, 0...","[-0.018036143854260445, 0.03526167571544647, 0..."
4,"Abscess of the Lung: Cough with phlegm or pus,...","[0.0018866059836000204, -0.03444769233465195, ...","[0.0018866059836000204, -0.03444769233465195, ..."


# Chain-of-Thought Model 1

In [7]:
import InstaDiagnose_1

In [10]:
doctor_1 = InstaDiagnose_1.Doctor(key)

doctor_1_full_histories = []
doctor_1_diagnoses_lists = []

In [11]:
doctor_1_full_histories, doctor_1_diagnoses_lists = run(0, doctor_1, doctor_1_full_histories, doctor_1_diagnoses_lists)

Patient 1
Symptoms: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scurring a lot.
Disease: acne

Patient: Hello, Doctor. I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scarring a lot.

Doctor response: {
  "if_continue": 1,
  "illness_list": ["Acne Vulgaris", "Folliculitis", "Impetigo", "Contact Dermatitis"],
  "response": "Thank you for sharing that information. Could you let me know if you're experiencing any additional symptoms like itching, swelling, or fever? This information can help improve the accuracy of the diagnosis."
}
Doctor response: {
  "if_continue": 1,
  "illness_list": ["Acne", "Folliculitis", "Impetigo", "Rosacea"],
  "response": "Thank you for sharing these details about your skin condition. To improve the accuracy of the diagnosis, could you please tell me if you're experiencing any additional symptoms such as it

In [12]:
df_model1_output = pd.DataFrame(data={'disease':df['output_text'].values, 'diagnosis lists':doctor_1_diagnoses_lists, 'conversations':doctor_1_full_histories})
df_model1_output.to_csv('output_model1.csv')

In [13]:
df_model1_output

,disease,diagnosis lists,conversations
0,acne,"[Acne, Folliculitis, Contact Dermatitis, Eczema]","[Patient: Hello, Doctor. I've been having a re..."
1,acne,"[Acne Vulgaris, Contact Dermatitis, Rosacea, F...","[Patient: Hi, doctor. I've just developed a se..."
2,allergy,"[Allergic Contact Dermatitis, Atopic Dermatiti...","[Patient: Hello, Doctor. My skin is itchy and ..."
3,allergy,"[Allergic Rhinitis, Sinusitis, Angioedema, Com...","[Patient: Hello, doctor. I have a sore throat ..."
4,arthritis,"[Rheumatoid Arthritis, Fibromyalgia, Polymyalg...","[Patient: Hello, I've been feeling really weak..."
5,arthritis,"[Rheumatoid Arthritis, Osteoarthritis, Fibromy...",[Patient: I've been having a lot of pain in my...
6,bronchial asthma,"[Asthma, Chronic Obstructive Pulmonary Disease...","[Patient: Hello, doctor. I've been having a ha..."
7,bronchial asthma,"[Chronic Obstructive Pulmonary Disease (COPD),...","[Patient: Hello, Doctor. I've been having a ha..."
8,cervical spondylosis,"[Multiple Sclerosis, Cervical Spondylosis, Chr...",[Patient: I have been having back pain for a w...
9,cervical spondylosis,"[Cervical Radiculopathy, Cervical Spondylosis,...",[Patient: I've been having trouble with my nec...


# Chain-of-Thought Model 2

In [7]:
import InstaDiagnose_2

In [8]:
doctor_2 = InstaDiagnose_2.Doctor(key)

In [9]:
doctor_2_full_histories, doctor_2_diagnoses_lists = [], []

In [10]:
doctor_2_full_histories, doctor_2_diagnoses_lists = run(0, doctor_2, doctor_2_full_histories, doctor_2_diagnoses_lists)

Patient 1
Symptoms: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scurring a lot.
Disease: acne

Patient: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scarring a lot.

Doctor: To improve the accuracy of the diagnosis, could you tell me more about any other symptoms you are experiencing, your medical history, or if you are currently taking any medications? This will help in narrowing down the possibilities further.

Patient: I've been having pus-filled pimples and blackheads. My skin is also scarring a lot. I do not know about other medical history or medications.

Doctor: Thank you for providing this information. Since the similarity score is below the threshold, I would like to know if there are any additional symptoms you might be experiencing, such as itching, pain, or fever. Additionally, if there are any other relevant aspects

In [12]:
df_model2_output = pd.DataFrame(data={'disease':df['output_text'].values, 'diagnosis lists':doctor_2_diagnoses_lists, 'conversations':doctor_2_full_histories})
df_model2_output.to_csv('output_model2.csv')

In [14]:
df_model2_output.head()

,disease,diagnosis lists,conversations
0,acne,"[Acne, Folliculitis, Impetigo, Eczema]",[Patient: I've been having a really bad rash o...
1,acne,"[Acne, Folliculitis, Contact Dermatitis, Impet...","[Patient: Hello, I recently developed a severe..."
2,allergy,"[Contact Dermatitis, Allergic Reaction, Eczema...",[Patient: Hello! My skin is itchy and red. Som...
3,allergy,"[Allergic Rhinitis, Angioedema, Common Cold, S...",[Patient: I have a sore throat and I am sneezi...
4,arthritis,"[Rheumatoid Arthritis, Lupus, Polymyalgia Rheu...",[Patient: I've been feeling really weak in my ...


# RAG Model 2 

In [7]:
import RAG_Diagnose_2

In [8]:
doctor_RAG_2 = RAG_Diagnose_2.Doctor(key, df_embed)

# rag2_full_histories = []
# rag2_diagnoses_lists = []

In [9]:
rag2_full_histories = []
rag2_diagnoses_lists = []

In [11]:
rag2_full_histories, rag2_diagnoses_lists = run(0, doctor_RAG_2, rag2_full_histories, rag2_diagnoses_lists)

Patient 1
Symptoms: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scurring a lot.
Disease: acne

Patient: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scarring a lot.

Doctor: To better understand your symptoms, could you provide more information about any other symptoms you might be experiencing, your medical history, and any current medications you're taking? This can help narrow down the potential causes of your skin condition.

Patient: I've been having a really bad rash on my skin lately, with pus-filled pimples and blackheads. My skin has also been scarring a lot. I do not have more information.

Patient Symptoms: 1. Rash on the skin
2. Pus-filled pimples
3. Presence of blackheads
4. Skin scarring 

doctor RAG (wrong format): ```json
{
    "if_continue": 0,
    "illness_list": ["Acne", "Folliculitis", "Impetigo", "Hidradeniti

In [14]:
df_rag_model2_output = pd.DataFrame(data={'disease':df['output_text'].values, 'diagnosis lists':rag2_diagnoses_lists, 'conversations':rag2_full_histories})
df_rag_model2_output.to_csv('output_rag2_patient_input_only.csv')

In [15]:
df_rag_model2_output.head()

,disease,diagnosis lists,conversations
0,acne,"[Acne, Folliculitis, Impetigo, Hidradenitis Su...",[Patient: I've been having a really bad rash o...
1,acne,"[Contact Dermatitis, Acne Vulgaris, Folliculit...","[Patient: Hello, Doctor. I've just developed a..."
2,allergy,"[Allergic Reaction, Eczema, Contact Dermatitis...","[Patient: My skin is itchy and red. Sometimes,..."
3,allergy,"[Allergic Rhinitis, Angioedema, Sinusitis, Com...","[Patient: Hello, doctor. I have a sore throat ..."
4,arthritis,"[Rheumatoid Arthritis, Polymyalgia Rheumatica,...",[Patient: I've been feeling really weak in my ...


# Chain-of-Thought Model 3

### Initialize Doctor 3

In [7]:
import InstaDiagnose_3
doctor_3 = InstaDiagnose_3.Doctor(key=key, threshold=0.75)

# client = OpenAI(api_key=key)

In [8]:
doctor_3_full_histories = []
doctor_3_diagnoses_lists = []

### Run Test Dataset

In [9]:
doctor_3_full_histories, doctor_3_diagnoses_lists = run(0, doctor_3, doctor_3_full_histories, doctor_3_diagnoses_lists)

Patient 1
Symptoms: I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scurring a lot.
Disease: acne

Patient: Hi, doctor. I've been having a really bad rash on my skin lately. It's full of pus-filled pimples and blackheads. My skin has also been scarring a lot.

Doctor: I'm sorry to hear about the discomfort you're experiencing. Let's try to gather some more information to better understand your symptoms.

1. How long have you been experiencing this rash and these skin changes?
2. Have you noticed any triggers or anything that seems to make the rash worse, such as certain foods, stress, or skincare products?
3. Do you recall any recent changes in your skincare routine or use of new products?
4. Have you experienced any other symptoms, such as itching or pain?
5. Are you currently taking any medications or have a history of any skin conditions?

Understanding these details will help me provide a more accurate asse

### Results

In [10]:
df_model3_output = pd.DataFrame(data={'disease':df['output_text'].values, 'diagnosis lists':doctor_3_diagnoses_lists, 'conversations':doctor_3_full_histories})

In [11]:
df_model3_output.to_csv('output_model3.csv')

# RAG Model 3

In [6]:
import RAG_Diagnose_3

In [7]:
doctor_RAG_3 = RAG_Diagnose_3.Doctor(key=key, df_embed=df_embed, threshold=0.75)

rag3_full_histories = []
rag3_diagnoses_lists = []

In [11]:
rag3_full_histories, rag3_diagnoses_lists = run(len(rag3_diagnoses_lists), doctor_RAG_3, rag3_full_histories, rag3_diagnoses_lists)

Patient 10
Symptoms: I've been having trouble with my neck. It's been hurting, and I've been having trouble staying balanced. I've also been having numbness in my arms and legs.
Disease: cervical spondylosis

Patient: I've been having trouble with my neck. It's been hurting, and I've been having trouble staying balanced. I've also been having numbness in my arms and legs.

diagnose (wrong format): Based on the conversation history provided, I cannot determine any symptoms as there is no information. Please provide details about your symptoms so I can assist you better.
Doctor: I'm sorry to hear about your discomfort. To help narrow down the possible causes, could you please provide a bit more information? 

1. How long have you been experiencing these neck pain and balance issues?
2. Have you had any recent injuries or accidents that might have affected your neck or spine?
3. Are there specific activities or movements that worsen the pain?
4. Do you have any other medical conditions or

In [13]:
df_rag_model3_output = pd.DataFrame(data={'disease':df['output_text'].values, 'diagnosis lists':rag3_diagnoses_lists, 'conversations':rag3_full_histories})
df_rag_model3_output.head()

,disease,diagnosis lists,conversations
0,acne,"[Acne Vulgaris, Folliculitis, Impetigo, Perior...",[Patient: Hello! I've been having a really bad...
1,acne,"[Acne, Impetigo, Contact Dermatitis, Rosacea]",[Patient: I've just developed a severe rash on...
2,allergy,"[Allergic Reaction, Contact Dermatitis, Atopic...","[Patient: Hello, I've been experiencing itchy ..."
3,allergy,"[Allergic Rhinitis, Non-allergic Rhinitis, Con...",[Patient: Hello! I've been having a sore throa...
4,arthritis,"[Rheumatoid Arthritis, Ankylosing Spondylitis,...",[Patient: I've been feeling really weak in my ...


In [16]:
df_rag_model3_output.to_csv('output_rag3_patient_input.csv')

# Chatbot Interface

In [48]:
import gradio as gr
full_history, diagnoses_list, threshold = [], [], 1

def alternatingly_agree(message, history):
    global diagnoses_list
    global full_history

    full_history.append(f"Patient: {message}")

    _, diagnosis, response, full_history = doctor_RAG_3.ask_doctor(threshold, diagnoses_list, full_history)

    diagnoses_list.append(diagnosis)
    return response
    

gr.ChatInterface(
    fn=alternatingly_agree, 
    type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Patient Sypmtoms:  1. Fever
2. Sore throat
3. Wet cough with mucus
4. Fatigue
5. Loss of taste
6. Temperature of about 38 degrees Celsius 



### Real Cases

In [49]:
full_history

['Patient: I have fever and sore throat.',
 "Diagnosis: ['Common Cold', 'Influenza', 'Strep Throat', 'Mononucleosis']",
 'Doctor: Thank you for sharing your symptoms. To get a clearer picture of what might be going on, could you please answer a few more questions?\n\n1. How long have you been experiencing the fever and sore throat?\n2. Do you have any other symptoms like body aches, cough, or swollen glands?\n3. Have you been in close contact with anyone who has similar symptoms recently?\n4. Are you currently taking any medications or do you have any pre-existing medical conditions? \n\nThis information will help in identifying the possible causes of your symptoms more accurately.',
 "Patient: 1. It has been several days. 2.I aslo have a wet cough. 3. I don't remember. 4. I had Tylenol before asking you.",
 "Diagnosis: ['Common Cold', 'Influenza', 'Streptococcal Pharyngitis', 'COVID-19']",
 "Doctor: Thank you for providing more information. Let's try to get a clearer understanding of 

In [23]:
flu_full_history

['Patient: i have fever and sore throat.',
 "Diagnosis: ['Common Cold', 'Flu (Influenza)', 'Strep Throat', 'COVID-19']",
 "Doctor: I understand you're experiencing a fever and sore throat. Let's gather a bit more information to help narrow down the possibilities. \n\n- How long have you had these symptoms?\n- Do you have any other symptoms like a cough, runny nose, or body aches?\n- Have you been in contact with anyone who is sick recently?\n- Are you currently taking any medications or do you have any chronic health issues that we should consider?\n\nFeel free to provide any additional information you think might be helpful.",
 'Patient: 1.several days. 2. i aslo have coughs. 3. no idea. 4. no',
 "Diagnosis: ['Viral Pharyngitis', 'Influenza', 'Common Cold', 'Streptococcal Pharyngitis']",
 "Doctor: Thank you for the additional information. Let's gather some more details to help clarify your symptoms:\n\n1. Do you have any headaches, fatigue, or loss of taste or smell?\n2. Have you noti

In [20]:
migraine_full_history

['Patient: I have headache and my left temple area is hurt as well.',
 "Diagnosis: ['Migraine', 'Temporal Arteritis', 'Tension Headache', 'Cluster Headache']",
 "Doctor: I'm sorry to hear you're experiencing pain in your head and temple area. To gain a clearer understanding of your symptoms, could you provide more detail about the nature of your headache? \n\n- How long have you been experiencing this pain?\n- Have you noticed any triggers that might cause or worsen your symptoms, such as specific foods, stress, or lack of sleep?\n- Do you have any accompanying symptoms such as nausea, vision changes, or sensitivity to light or sound?\n- Are you currently taking any medications, either for this issue or any other reason?\n- Do you have a history of migraines or any other medical conditions?\n\nYour responses will help me better assess your situation.",
 'Patient: 1. I have been experiencing this pain for long time. 2. When I work for long time in front of computer, I feel this pain. 3.

In [12]:
covid_full_history

['Patient: Hello doctor, I have symptoms of vomiting and diarrhea.',
 "Diagnosis: ['Gastroenteritis', 'Food Poisoning', 'Viral Infection', 'Irritable Bowel Syndrome (IBS)']",
 'Doctor: Thank you for sharing your symptoms with me. To better understand your situation, could you please provide more details? \n\n1. When did the vomiting and diarrhea start? \n2. Have you eaten anything unusual or suspect recently?\n3. Are you experiencing any other symptoms, such as fever, abdominal pain, or fatigue?\n4. Do you have any history of gastrointestinal issues or are you currently taking any medications? \n\nYour responses will help in assessing your condition further.',
 "Patient: The symptoms started yesterday. I haven't eaten anything unusual or suspect. I do have fever and sore throat.",
 "Diagnosis: ['Gastroenteritis', 'Viral Infection (e.g., viral pharyngitis)', 'Food Poisoning', 'Stomach Flu (viral gastroenteritis)']",
 "Doctor: Based on the symptoms you've described, possible illnesses in

In [15]:
pneumonia_full_history

['Patient: Hello doctor, I have been coughing and having fever for a week. ',
 "Diagnosis: ['Common Cold', 'Influenza', 'Pneumonia', 'COVID-19']",
 "Doctor: I'm sorry to hear that you've been feeling unwell. Let's gather some more information to better understand your symptoms:\n\n1. Have you noticed any additional symptoms such as shortness of breath, chest pain, or fatigue?\n2. Have you had any recent exposure to anyone with similar symptoms or been in environments where viruses are spreading?\n3. Do you have any underlying medical conditions or are you currently on any medications?\n4. Have you experienced any improvement or worsening of symptoms, and have you tried any remedies or treatments so far?\n\nYour answers will help me narrow down the possibilities and provide better advice.",
 'Patient: I have symptom of shortness of breath and fatigue. I am not taking any medications.',
 "Diagnosis: ['COVID-19', 'Influenza', 'Pneumonia', 'Bronchitis']",
 "Doctor: Based on the symptoms yo

In [36]:
covid_full_history

['Patient: Hello doctor, I have symptoms of vomiting and diarrhea.',
 "Diagnosis: ['Gastroenteritis', 'Food Poisoning', 'Viral Infection', 'Irritable Bowel Syndrome (IBS)']",
 'Doctor: Thank you for sharing your symptoms with me. To better understand your situation, could you please provide more details? \n\n1. When did the vomiting and diarrhea start? \n2. Have you eaten anything unusual or suspect recently?\n3. Are you experiencing any other symptoms, such as fever, abdominal pain, or fatigue?\n4. Do you have any history of gastrointestinal issues or are you currently taking any medications? \n\nYour responses will help in assessing your condition further.',
 "Patient: The symptoms started yesterday. I haven't eaten anything unusual or suspect. I do have fever and sore throat.",
 "Diagnosis: ['Gastroenteritis', 'Viral Infection (e.g., viral pharyngitis)', 'Food Poisoning', 'Stomach Flu (viral gastroenteritis)']",
 "Doctor: Based on the symptoms you've described, possible illnesses in

In [39]:
def clean_history(history):
    doctor_patient_conversatin = []

    for i in range(0, len(history), 3):
        doctor_patient_conversatin.append(history[i])
        try:
            doctor_patient_conversatin.append(history[i+2])
        except:
            pass

    return doctor_patient_conversatin

cleaned_histories = []
for h in [flu_full_history, migraine_full_history, covid_full_history, pneumonia_full_history]:
    cleaned_histories.append(clean_history(h))

In [43]:
df_real_case = pd.DataFrame(data={'disease':['influenza', 'migraine', 'covid', 'pneumonia'], 'conversation':cleaned_histories})
df_real_case.to_csv('output_real_case.csv')
df_real_case.head()

,disease,conversation
0,influenza,"[Patient: i have fever and sore throat., Docto..."
1,migraine,[Patient: I have headache and my left temple a...
2,covid,"[Patient: Hello doctor, I have symptoms of vom..."
3,pneumonia,"[Patient: Hello doctor, I have been coughing a..."
